# 05 - Agente com Objetivo e Modelo Interno usando LLM 🤖🎯

Neste notebook, vamos criar um **agente com objetivo** que:

✅ Tem um objetivo explícito (ex.: limpar sujeira).  
✅ Usa um **modelo interno** baseado em um **LLM (Large Language Model)** para interpretar o ambiente e decidir a ação.  
✅ Não guarda histórico de passos anteriores (sem memória de longo prazo ainda!).

---

## 📦 Preparação do ambiente

Para rodar este notebook, certifique-se de ter instalado:

```bash
pip install torch transformers


In [1]:
import torch
print(torch.__version__)

2.7.0+cpu


### Exemplo prático

Imagine um aspirador robô:

Objetivo: limpar toda a sujeira.

Modelo interno: um LLM que interpreta a descrição do ambiente.

Ação: o LLM gera a ação mais adequada.

In [2]:
from transformers import pipeline

class AgenteObjetivoModeloLLM:
    def __init__(self, objetivo="clean"):
        self.objetivo = objetivo
        # Usando DistilGPT2 para geração simples de ações
        self.gerador = pipeline("text-generation", model="distilgpt2")

    def agir(self, modelo_ambiente):
        """
        modelo_ambiente: dicionário descrevendo o ambiente (em inglês!)
        Exemplo:
            {
                "front": "free",
                "left": "dirt",
                "right": "obstacle"
            }
        """
        # Descreve o ambiente em linguagem natural
        descricao = f"Environment state: front={modelo_ambiente['front']}, left={modelo_ambiente['left']}, right={modelo_ambiente['right']}."

        # Prompt para o LLM
        prompt = (
            f"You are a cleaning robot. Your goal is to {self.objetivo} the dirt.\n"
            f"{descricao}\n"
            f"What action should you take? Respond with a single word action."
        )

        resposta = self.gerador(
            prompt,
            max_new_tokens=20,
            truncation=True,
            num_return_sequences=1
        )[0]["generated_text"]

        return resposta.strip()

# Instanciando o agente
agente = AgenteObjetivoModeloLLM()


c:\Users\vinicius\Documents\Projetos\AI-Agentes-Study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


### Testando o agente com exemplos de ambiente

In [3]:
# Exemplos de modelos internos do ambiente
modelos_ambiente = [
    {"front": "free", "left": "dirt", "right": "obstacle"},
    {"front": "dirt", "left": "free", "right": "free"},
    {"front": "obstacle", "left": "dirt", "right": "free"},
    {"front": "free", "left": "free", "right": "free"}
]

# Rodando a simulação
for i, modelo in enumerate(modelos_ambiente):
    acao = agente.agir(modelo)
    print(20*'@')
    print(f"Step {i+1}: Environment={modelo} => Action={acao}")
    print()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


@@@@@@@@@@@@@@@@@@@@
Step 1: Environment={'front': 'free', 'left': 'dirt', 'right': 'obstacle'} => Action=You are a cleaning robot. Your goal is to clean the dirt.
Environment state: front=free, left=dirt, right=obstacle.
What action should you take? Respond with a single word action. Use your voice to speak.
If you want to make a more informed decision, click here.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


@@@@@@@@@@@@@@@@@@@@
Step 2: Environment={'front': 'dirt', 'left': 'free', 'right': 'free'} => Action=You are a cleaning robot. Your goal is to clean the dirt.
Environment state: front=dirt, left=free, right=free.
What action should you take? Respond with a single word action. If you are unsure, you can ask our Customer Service team to provide a brief description of what to



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


@@@@@@@@@@@@@@@@@@@@
Step 3: Environment={'front': 'obstacle', 'left': 'dirt', 'right': 'free'} => Action=You are a cleaning robot. Your goal is to clean the dirt.
Environment state: front=obstacle, left=dirt, right=free.
What action should you take? Respond with a single word action.
Please feel free to share your thoughts and suggestions and ideas with other cleaning robots using the comment below

@@@@@@@@@@@@@@@@@@@@
Step 4: Environment={'front': 'free', 'left': 'free', 'right': 'free'} => Action=You are a cleaning robot. Your goal is to clean the dirt.
Environment state: front=free, left=free, right=free.
What action should you take? Respond with a single word action.
In the main area, you may want to look at a local resident's home.
Your



In [9]:
from transformers import pipeline

class AgenteObjetivoModeloClassifier:
    def __init__(self, objetivo="clean"):
        self.objetivo = objetivo
        # Usando DistilBERT treinado para classificação de sentimentos (ótimo em inglês!)
        self.classificador = pipeline(
            "text-classification",
            model="distilbert-base-uncased-finetuned-sst-2-english"
        )

    def agir(self, modelo_ambiente):
        """
        modelo_ambiente: dicionário descrevendo o ambiente (em inglês!)
        Exemplo:
            {
                "front": "free",
                "left": "dirt",
                "right": "obstacle"
            }
        """
        # Descreve o ambiente em linguagem natural
        # descricao = (
        #     f"Environment state: front={modelo_ambiente['front']}, "
        #     f"left={modelo_ambiente['left']}, right={modelo_ambiente['right']}."
        # )
        descricao = (
        f"I feel this environment is: front={modelo_ambiente['front']}, "
        f"left={modelo_ambiente['left']}, right={modelo_ambiente['right']}."
    )

        # Classificação com o LLM
        resultado = self.classificador(descricao)[0]
        print(resultado)

        # Mapeia a classificação para ação
        if resultado["label"] == "POSITIVE":
            return "move forward"
        else:
            return "avoid"

# Instanciando o agente
agente = AgenteObjetivoModeloClassifier()

Device set to use cpu


In [10]:
# Exemplos de modelos internos do ambiente
modelos_ambiente = [
    {"front": "free", "left": "dirt", "right": "obstacle"},
    {"front": "dirt", "left": "free", "right": "free"},
    {"front": "obstacle", "left": "dirt", "right": "free"},
    {"front": "free", "left": "free", "right": "free"}
]

# Simulação
for i, modelo in enumerate(modelos_ambiente):
    acao = agente.agir(modelo)
    print(f"Step {i+1}: Environment={modelo} => Action={acao}")


{'label': 'NEGATIVE', 'score': 0.9828706979751587}
Step 1: Environment={'front': 'free', 'left': 'dirt', 'right': 'obstacle'} => Action=avoid
{'label': 'NEGATIVE', 'score': 0.9895250797271729}
Step 2: Environment={'front': 'dirt', 'left': 'free', 'right': 'free'} => Action=avoid
{'label': 'NEGATIVE', 'score': 0.9656710028648376}
Step 3: Environment={'front': 'obstacle', 'left': 'dirt', 'right': 'free'} => Action=avoid
{'label': 'NEGATIVE', 'score': 0.9822536110877991}
Step 4: Environment={'front': 'free', 'left': 'free', 'right': 'free'} => Action=avoid


In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Carregar variáveis de ambiente
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Inicializar cliente da OpenAI
client = OpenAI(api_key=openai_api_key)

class AgenteObjetivoModeloAPI:
    def __init__(self, objetivo="clean"):
        self.objetivo = objetivo

    def agir(self, modelo_ambiente):
        """
        modelo_ambiente: dicionário descrevendo o ambiente (em inglês!)
        Exemplo:
            {
                "front": "free",
                "left": "dirt",
                "right": "obstacle"
            }
        """
        descricao = (
            f"Environment state: front={modelo_ambiente['front']}, "
            f"left={modelo_ambiente['left']}, right={modelo_ambiente['right']}."
        )

        prompt = (
            f"You are a cleaning robot. Your goal is to {self.objetivo} the dirt.\n"
            f"{descricao}\n"
            f"What action should you take? Respond with a single word action."
        )

        resposta = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are an intelligent cleaning robot."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )

        # Pega a resposta final
        acao = resposta.choices[0].message.content.strip()
        return acao

# Teste do agente
agente = AgenteObjetivoModeloAPI()

modelos_ambiente = [
    {"front": "free", "left": "dirt", "right": "obstacle"},
    {"front": "dirt", "left": "free", "right": "free"},
    {"front": "obstacle", "left": "dirt", "right": "free"},
    {"front": "free", "left": "free", "right": "free"}
]

for i, modelo in enumerate(modelos_ambiente):
    acao = agente.agir(modelo)
    print(f"Step {i+1}: Environment={modelo} => Action={acao}")


Step 1: Environment={'front': 'free', 'left': 'dirt', 'right': 'obstacle'} => Action=LEFT
Step 2: Environment={'front': 'dirt', 'left': 'free', 'right': 'free'} => Action=Clean
Step 3: Environment={'front': 'obstacle', 'left': 'dirt', 'right': 'free'} => Action=Turn
Step 4: Environment={'front': 'free', 'left': 'free', 'right': 'free'} => Action=Move forward.
